# Trigrams

In [ ]:
from nltk.collocations import (
    BigramCollocationFinder, BigramAssocMeasures,
    TrigramCollocationFinder, TrigramAssocMeasures,
    )

In [ ]:
trigram_measures = TrigramAssocMeasures()
trigram_finder = TrigramCollocationFinder.from_words(testcorpus)

In [ ]:
# bigram_measures = nltk.collocations.BigramAssocMeasures()
# bigram_finder = BigramCollocationFinder.from_words(corpus)

In [ ]:
# finder.apply_freq_filter(20)
stops = set((
    ' ? . , ! den de det dette at om har med er jo det'
    ' å de som av the ikke få bli for en et eit til'
    ' sier han ved blant mer '
    ).split() + ['']) 
sidestops = set('og i for'.split())

def sidestop_filter(*w):
    return w[0] in sidestops or w[-1] in sidestops

trigram_finder.apply_freq_filter(3)
trigram_finder.apply_word_filter(str.isdigit)
trigram_finder.apply_word_filter(lambda w: w.lower() in stops)
trigram_finder.apply_ngram_filter(sidestop_filter)

In [ ]:
best_pmi = set(trigram_finder.nbest(trigram_measures.pmi, 5000))
def in_best_pmi(*trigram):
    return trigram not in best_pmi
trigram_finder.apply_ngram_filter(in_best_pmi)

In [ ]:
#trigram_finder.score_ngrams(trigram_measures.likelihood_ratio)
trigram_finder.score_ngrams(trigram_measures.pmi)
#trigram_finder.score_ngrams(trigram_measures.likelihood_ratio)

# Single word keywords

In [ ]:
from nltk.stem.snowball import NorwegianStemmer
from collections import Counter

In [ ]:
from apps.stories.keywords import *

In [ ]:
corpus_dict = corpus_dict()

In [ ]:
len(corpus_dict)

In [ ]:
def random_story():
    return Story.objects.order_by('?').first()


In [ ]:

def build_keywords(self, corpus_dict, limit=10):
    """build keywords"""
    stems = stem_words(self.get_words())
    valid_words = (word for word in stems if word in corpus_dict)
    word_counts = Counter(valid_words).items()
    keywords = ((w, c, corpus_dict[w]) for w, c in word_counts)
    best = sorted(
        keywords,
        key=lambda kw: kw[1]/kw[2],
        reverse=True
    )[:limit]
    #print(keywords, best)
    return best
    #return OrderedDict(
    #    (key, val) for key, val in keywords.items() if key in best)

#Story.build_keywords = build_keywords
story = random_story()
story.build_keywords(corpus_dict, 20)


In [1]:
import json
with open('corpusdict.json', 'r') as fp:
    corpus_dict = json.load(fp)

In [31]:
count = 0
for story in Story.objects.filter(keywords=''):
    keywords = story.build_keywords(corpus_dict, 5)
    #story.keywords = ' '.join(kw[0] for kw in keywords)
    #story.save(update_fields=['keywords'])
    if count%10 == 0:
        print(count)
        print('\n', story.title, '\n', story.keywords.replace(' ', '\n'))
    count += 1

0

 Urealiserte tusenårsmål 
 tusenårsmål
proteksjonistisk
planøkonomi
indern
barnedød
10

 Mytekalenderen 
 sidrump
markedsretting
korrumper
kjært
karriereveiledning
20

 Hvilke universitetsplaner? 
 ssb-undersøk
universitetssatsing
høyskoleutdanning
korssjø
høgskolestyr
30

 Velg Helsetjenesten 
 fastlegereform
føflekk
allslag
legehjelp
legetjenest
40

 Grøtglad sosialist 
 moxn
begås
karaok
kjepphest
snusfornuft
50

 Lykkelig gift? 
 oslo-borger
rikspress
pressgrupp
kraftigst
skral
60

 Nuddelpolitikk 
 kåss
hvileput
romantisering
lønnsutvikling
boligkris
70

 Oppskrytt 
 pedagogikk-kur
babbel
høytsvev
pano
relevans
80

 En boks med mye rart i 
 finalerund
exaggeration
romanc
taktløs
kir
90

 Kampen om disco 
 dimitri
playboy
fanskar
greker
rener
100

 Massiv kritikk mot fjerning av reisestipendet 
 østerås
sundtoft
formålsparagraf
fjerning
underskrift
110

 Tar jazzarven videre 
 jazzscen
rypdal
konsertseri
konservatori
jazzmiljø
120

 Om «Skarpladd historieforteller» i Universitas

In [ ]:
with open('corpusdict.json', 'w') as fp:
    json.dump(corpus_dict, fp)

In [4]:
story

<Story: 2004-04-19: Fantastisk klamt>

In [5]:
story

<Story: 2004-04-19: Fantastisk klamt>

In [6]:
story.keywords

'motfestival\nnova-festival\nkommersialism\nnaim\nslant'

In [29]:
from collections import defaultdict

def related_stories(self):
    scores = defaultdict(lambda:0)
    keywords =  self.keywords.split('\n')
    for keyword in keywords:
        candidates = Story.objects.exclude(pk=self.pk).filter(keywords__icontains=keyword)
        for candidate in candidates:
            score = (
                10 - 10*(self.keywords.index(keyword) / len(self.keywords)) +
                10 -10*(candidate.keywords.index(keyword) / len(candidate.keywords) )
                ) 
            # print(candidate.title, candidate.keywords, score, keyword)
            scores[candidate.pk] += score
    pks = sorted(scores.keys(), key=lambda k: scores[k], reverse=True)[:3]
    return Story.objects.filter(pk__in=pks)

Story.related_stories = related_stories
story.related_stories()

[<Story: 2010-01-26: Nytt til folket>, <Story: 2006-11-21: Alt er feil og føkka>, <Story: 2009-04-28: Dassrock>]

In [26]:
sorted(scores.keys(), key=lambda k: scores[k], reverse=True)[:3]

[53321, 54346, 53699]

In [30]:
story

<Story: 2004-04-19: Fantastisk klamt>

In [50]:

def try_json_first(filename):
    """decorator that memoizes json data if called with no args"""
    def decorator_wrapper(func):
        def func_wrapper(*args, **kwargs):
            if args or kwargs:
                return func(*args, **kwargs)
            try:
                with open(filename, 'r') as fp:
                    return json.load(fp)
            except FileNotFoundError:
                data = func()
                with open(filename, 'w') as fp:
                    json.dump(data, fp)
                return data
        return func_wrapper
    return decorator_wrapper

@try_json_first('lureri.json')
def lureri(*args):
    if not args:
        return {          'thing': 42,}
    return {'args': args}

import os 
#os.remove('lureri.json')
print(lureri('heisann'))
print(lureri('hoppsann'))

{'args': ('heisann',)}
{'args': ('hoppsann',)}


In [52]:

def try_json_first(filename):
    """decorator that memoizes json data if called with no args"""
    def decorator_wrapper(func):
        def func_wrapper(*args, **kwargs):
            if args or kwargs:
                return func(*args, **kwargs)
            try:
                with open(filename, 'r') as fp:
                    return json.load(fp)
            except FileNotFoundError:
                data = func()
                with open(filename, 'w') as fp:
                    json.dump(data, fp)
                return data
        return func_wrapper
    return decorator_wrapper

@try_json_first('lureri.json')
def lureri(*args):
    if not args:
        return {          'thing': 42,}
    return {'args': args}

import os 
#os.remove('lureri.json')
print(lureri('heisann'))
print(lureri())
print(lureri('hoppsann'))

{'args': ('heisann',)}
{'thing2': 42, 'thing': 42}
{'args': ('hoppsann',)}
